In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

if os.path.isfile("vids.pkl"):
    vids = pd.read_pickle("vids.pkl")
    
#randomise rows
vids = vids.sample(frac=1).reset_index(drop=True)
le = LabelEncoder()
le.fit(vids["Broad Tag"].unique());

array(['Comedy', 'Gaming', 'Music', 'News', 'Sports', 'Tutorial'],
      dtype=object)

In [10]:
vids["TagCode"] = le.transform(vids["Broad Tag"])

In [11]:
#we will use title and broad tag first

training, test = train_test_split(vids, test_size=0.33, random_state=42)
train_x = training["title"]
train_y = training["TagCode"]

test_x = test["title"]
test_y = test["TagCode"]

In [12]:
vect = CountVectorizer()
tr_x = vect.fit_transform(train_x)
tes_x = vect.transform(test_x)

In [49]:
tr_x = tr_x.toarray()

AttributeError: 'numpy.ndarray' object has no attribute 'toarray'

In [50]:
tes_x = tes_x.toarray()

In [44]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1,719)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(6)
])

In [45]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [46]:
model.fit(tr_x, train_y, epochs=10)

Epoch 1/10
6/6 [==============================] - 0s 8ms/step - loss: 1.7725 - accuracy: 0.2121
Epoch 2/10
6/6 [==============================] - 0s 2ms/step - loss: 1.5831 - accuracy: 0.7030
Epoch 3/10
6/6 [==============================] - 0s 2ms/step - loss: 1.4196 - accuracy: 0.9455
Epoch 4/10
6/6 [==============================] - 0s 2ms/step - loss: 1.2690 - accuracy: 0.9879
Epoch 5/10
6/6 [==============================] - 0s 2ms/step - loss: 1.1221 - accuracy: 0.9879
Epoch 6/10
6/6 [==============================] - 0s 2ms/step - loss: 0.9807 - accuracy: 0.9939
Epoch 7/10
6/6 [==============================] - 0s 2ms/step - loss: 0.8430 - accuracy: 1.0000
Epoch 8/10
6/6 [==============================] - 0s 2ms/step - loss: 0.7146 - accuracy: 1.0000
Epoch 9/10
6/6 [==============================] - 0s 2ms/step - loss: 0.5970 - accuracy: 1.0000
Epoch 10/10
6/6 [==============================] - 0s 2ms/step - loss: 0.4918 - accuracy: 1.0000


In [52]:
test_loss, test_acc = model.evaluate(tes_x, test_y, verbose=2)
print(test_acc)

3/3 - 0s - loss: 0.7096 - accuracy: 0.9268
0.9268292784690857


In [54]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(tes_x);

In [82]:
le.inverse_transform([np.argmax(predictions[2])]);
predictions[0]

array([0.05714076, 0.08008232, 0.21827549, 0.1154823 , 0.41258016,
       0.11643892], dtype=float32)

In [98]:
le.inverse_transform([np.argmax(probability_model.predict(vect.transform(["how to swim"]).toarray()))])

array(['Tutorial'], dtype=object)

In [99]:
hmm = probability_model.predict(vect.transform(["how to swim"]).toarray())
hmm

array([[0.08862431, 0.12510645, 0.19499986, 0.09144052, 0.10472817,
        0.39510074]], dtype=float32)